In [74]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
#from prince import MCA

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

import matplotlib.pyplot as plt

In [19]:
data = pd.read_csv('Data/TrainingData.csv')

cols = data.columns

print (data.shape)

data.head()

(44102, 130)


,UNIQUE_ID,Overall_Rating,Technical_Skills,Teamwork,Customer_Service,Hire_Again,High_Performer,Protected_Group,Retained,SJ_Most_1,...,PScale12_Q1,PScale12_Q2,PScale12_Q3,PScale12_Q4,PScale13_Q1,PScale13_Q2,PScale13_Q3,PScale13_Q4,PScale13_Q5,split
0,245021089,3.0,3.0,4.0,4.0,4.0,0.0,0.0,1,3.0,...,1.0,1.0,3.0,4.0,1.0,2.0,3.0,2.0,1.0,train
1,245181465,5.0,5.0,5.0,5.0,5.0,1.0,1.0,0,3.0,...,1.0,1.0,4.0,4.0,2.0,1.0,4.0,4.0,4.0,train
2,229682665,3.0,3.0,3.0,3.0,4.0,0.0,1.0,0,2.0,...,1.0,1.0,4.0,4.0,1.0,1.0,4.0,4.0,4.0,train
3,245174982,4.0,4.0,4.0,4.0,5.0,1.0,0.0,1,2.0,...,1.0,1.0,2.0,4.0,1.0,1.0,3.0,3.0,3.0,train
4,244979030,2.0,2.0,3.0,2.0,3.0,0.0,0.0,1,3.0,...,2.0,1.0,4.0,4.0,2.0,1.0,4.0,3.0,2.0,train


In [28]:
pscore = data[cols[cols.str.contains('PScale')]]
pscore.head()

,PScale01_Q1,PScale01_Q2,PScale01_Q3,PScale01_Q4,PScale02_Q1,PScale02_Q2,PScale02_Q3,PScale02_Q4,PScale03_Q1,PScale03_Q2,...,PScale11_Q4,PScale12_Q1,PScale12_Q2,PScale12_Q3,PScale12_Q4,PScale13_Q1,PScale13_Q2,PScale13_Q3,PScale13_Q4,PScale13_Q5
0,4.0,1.0,1.0,4.0,3.0,1.0,4.0,4.0,1.0,3.0,...,1.0,1.0,1.0,3.0,4.0,1.0,2.0,3.0,2.0,1.0
1,4.0,1.0,1.0,4.0,4.0,1.0,4.0,4.0,1.0,4.0,...,1.0,1.0,1.0,4.0,4.0,2.0,1.0,4.0,4.0,4.0
2,4.0,1.0,1.0,4.0,4.0,1.0,4.0,4.0,1.0,4.0,...,4.0,1.0,1.0,4.0,4.0,1.0,1.0,4.0,4.0,4.0
3,3.0,3.0,3.0,3.0,4.0,1.0,4.0,3.0,1.0,4.0,...,2.0,1.0,1.0,2.0,4.0,1.0,1.0,3.0,3.0,3.0
4,2.0,3.0,3.0,2.0,3.0,2.0,3.0,2.0,1.0,3.0,...,4.0,2.0,1.0,4.0,4.0,2.0,1.0,4.0,3.0,2.0


In [79]:
q_filter = [('PScale01', [2,3], 5), ('PScale02', [2], 5), ('PScale03', [1], 5), ('PScale04', [1], 5),
            ('PScale05', [], 5), ('PScale06', [3,6], 5), ('PScale07', [2], 5), ('PScale08', [1,3], 5),
            ('PScale09', [2], 5), ('PScale10', [3,4], 5), ('PScale11', [2,3], 5), 
            ('PScale12', [3,4], 5), ('PScale13', [3,4], 5)]


def pscore_rescale (pscore, skipna=True):
    old_cols = []
    pscore_rev = pscore.copy()
    for nm, cols, scale in q_filter:
        for col in cols:
            col_name = f'{nm}_Q{col}'
            col = scale - pscore[col_name]
            pscore_rev.loc[:,col_name] = col
        pscore_rev[nm] = pscore_rev.filter(regex=f'{nm}_*').mean(axis=1, skipna=skipna)

    pscore_rev = pscore_rev.loc[:,~pscore_rev.columns.str.contains('_')]
    return pscore_rev
         

In [81]:
ps = pscore_rescale(pscore, skipna=False)

ps.head()

,PScale01,PScale02,PScale03,PScale04,PScale05,PScale06,PScale07,PScale08,PScale09,PScale10,PScale11,PScale12,PScale13
0,4.0,3.75,3.50,1.50,3.75,2.833333,2.0,3.25,1.0,1.0,1.00,1.25,1.8
1,4.0,4.00,4.00,1.00,3.25,2.500000,1.0,2.25,1.0,1.0,1.00,1.00,1.8
2,4.0,4.00,4.00,2.50,1.75,3.000000,1.0,1.00,1.0,1.0,2.50,1.00,1.6
3,2.5,3.75,3.75,2.00,3.00,2.500000,1.0,2.00,1.0,2.0,2.25,1.50,1.8
4,2.0,2.75,3.00,3.25,4.00,3.333333,2.0,2.00,1.0,2.5,4.00,1.25,1.6
